# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [3]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# get total number of rows 
print(len(full_data_rows_list))
# uncomment to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. the Apache Cassandra coding. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# create a connection to the database
from cassandra.cluster import Cluster
try:
    cluster = Cluster()
    session = cluster.connect() 
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify  
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('sparkify') #Connect to our Keyspace
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




   # Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

### Since our data is looking for the sessionId it will be the partition key. then we will add itemInSession as a clustering column to make sure the Primary Key is unique.

`Table Name: song_history_session
column 1: session_id
column 2: item_in_session
column 3: artist_name
Column 4: song_title
Column 5: song_length
PRIMARY KEY(session_id, item_in_session)`


In [11]:
query = "CREATE TABLE IF NOT EXISTS song_history_session "
query = query + "(session_id int, item_in_session int, artist_name varchar, song_title varchar, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [12]:
# set up the CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_history_session (session_id, item_in_session, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            #Assign the column element for each column in the INSERT statement.
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)                                       

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
# SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, song_title, song_length FROM song_history_session WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


   # Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### Since our data is looking for the userid and sessionid this will be composite partition key. then we will add  item_in_session as a clustering column to sort the data and make sure the Primary Key is unique. 

`Table Name: song_user_session
column 1: user_id
column 2: session_id
column 3: item_in_session
Column 4: artist_name
Column 5: song_title
Column 6: user_fname
Column 7: user_lname
PRIMARY KEY((user_id, session_id), item_in_session))`


In [14]:
query = "CREATE TABLE IF NOT EXISTS song_user_session "
query = query + "(user_id int, session_id int, item_in_session int, artist_name varchar, song_title varchar, user_fname varchar, user_lname varchar, PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [15]:
# set up the CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user_session(user_id, session_id, item_in_session, artist_name, song_title, user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            #Assign the column element for each column in the INSERT statement.
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        except Exception as e:
            print(e)          

In [17]:
# SELECT statement to verify the data was entered into the table
query = """SELECT artist_name, song_title, user_fname, user_lname 
          FROM song_user_session 
          WHERE user_id = 10 AND session_id = 182 
          ORDER BY item_in_session"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.user_fname, row.user_lname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Since our data is looking for the song title it will be the partition key. then we will add user_id as a clustering column to make sure the Primary Key is unique.

`Table Name: song_user
column 1: song_title
column 2: user_id
column 3: user_fname
Column 4: user_lname
PRIMARY KEY(song_title, user_id)`


In [18]:
query = "CREATE TABLE IF NOT EXISTS song_user"
query = query + "(song_title varchar, user_id int, user_fname varchar, user_lname varchar, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                       

In [19]:
# set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_user(song_title, user_id, user_fname, user_lname)"
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            #Assign the column element for each column in the INSERT statement.
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)          

In [20]:
# the SELECT statement to verify the data was entered into the table
query = "SELECT user_fname, user_lname FROM song_user WHERE song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_fname, row.user_lname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [21]:
query = "drop table song_history_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_user_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()